<a href="https://colab.research.google.com/github/Raman-kr1/AMEX-Campus-challenge-2024/blob/main/Correlation_Matrix_Amexx_r3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline


In [ ]:
# Load the dataset from the csv file using pandas
# best way is to mount the drive on colab and
# copy the path for the csv file
file_path = '/content/without string book1.xlsx'
data = pd.read_excel(file_path)


In [ ]:
# Grab a peek at the data
data.head()



,unique_identifier,risk_score_1,bureau_score,limit,income,old_open_trd_ms,no_open_trd,no_trd,debt_cap,basic_max_trd_tnr,...,merchant2_cat,merchant3,merchant3_amt,merchant3_ten,merchant3_cat,risk_score_8,risk_score_9,risk_score_10,risk_score_11,default_ind
0,116356,0.0,800,5000,45000,0,0,0,207.890000,0,...,0,0,0.0,0,0,0.000,0.0000,0.012,0.2787,0
1,110872,0.1,775,13000,100000,352,11,18,40677.000000,351,...,0,0,0.0,0,0,0.999,0.0092,0.070,0.2275,0
2,158686,5.6,900,2000,100000,1,1,1,59921.616667,1,...,0,0,0.0,0,0,0.002,0.0015,0.035,0.2787,1
3,148772,0.0,800,5000,170000,0,0,0,148803.226100,0,...,1,0,0.0,0,0,0.252,0.0253,0.131,0.1175,0
4,158473,0.1,775,10000,40000,267,3,18,26017.000000,182,...,0,0,0.0,0,0,0.000,0.0000,0.000,0.2035,0


In [ ]:
print(data.head())
print(data.info())

   unique_identifier  risk_score_1  bureau_score  limit  income  \
0             116356           0.0           800   5000   45000   
1             110872           0.1           775  13000  100000   
2             158686           5.6           900   2000  100000   
3             148772           0.0           800   5000  170000   
4             158473           0.1           775  10000   40000   

   old_open_trd_ms  no_open_trd  no_trd       debt_cap  basic_max_trd_tnr  \
0                0            0       0     207.890000                  0   
1              352           11      18   40677.000000                351   
2                1            1       1   59921.616667                  1   
3                0            0       0  148803.226100                  0   
4              267            3      18   26017.000000                182   

   ...  merchant2_cat  merchant3  merchant3_amt  merchant3_ten  merchant3_cat  \
0  ...              0          0            0.0      

In [ ]:
# Print the shape of the data
# data = data.sample(frac = 0.1, random_state = 48)
print(data.shape)
print(data.describe())


(62484, 54)
       unique_identifier  risk_score_1  bureau_score         limit  \
count       62484.000000  62484.000000  62484.000000  62484.000000   
mean       131242.500000      1.890106    772.460550  10731.355227   
std         18037.721447      3.294003     71.644113  10303.905843   
min        100001.000000      0.000000    525.000000    500.000000   
25%        115621.750000      0.000000    725.000000   2000.000000   
50%        131242.500000      0.200000    775.000000   6000.000000   
75%        146863.250000      2.700000    800.000000  15000.000000   
max        162484.000000     77.400000    900.000000  50000.000000   

               income  old_open_trd_ms   no_open_trd        no_trd  \
count    62484.000000     62484.000000  62484.000000  62484.000000   
mean    127925.981051       104.247503      3.834086      7.944882   
std     134003.357926       138.466501      5.519016     11.758840   
min       1000.000000         0.000000      0.000000      0.000000   
25%    

Imbalance in the data

In [ ]:
# find number of fraud cases;
fraud = data[data['default_ind'] == 1]
valid = data[data['default_ind'] == 0]
outlier_fraction = len(fraud)/float(len(valid))
print(outlier_fraction)
print('Fraud Cases: {}'.format(len(data[data['default_ind'] == 1])))
print('Valid Transactions: {}'.format(len(data[data['default_ind'] == 0])))

0.005843434587337615
Fraud Cases: 363
Valid Transactions: 62121


stat for fraudulent transactions

In [ ]:
print('incomedetails of the fraudulent transaction')
fraud.income.describe()

incomedetails of the fraudulent transaction


,income
count,363.000000
mean,102768.595041
std,111565.858483
min,15000.000000
25%,55000.000000
50%,75000.000000
75%,110000.000000
max,1000000.000000


In [ ]:
# valid transactions;
print('income details of the valid transaction')
valid.income.describe()

income details of the valid transaction


,income
count,62121.000000
mean,128072.986591
std,134110.320122
min,1000.000000
25%,55000.000000
50%,90000.000000
75%,150000.000000
max,1000000.000000


In [ ]:
corr_matrix = data.corr()
print(corr_matrix)


                       unique_identifier  risk_score_1  bureau_score  \
unique_identifier               1.000000     -0.048849      0.005555   
risk_score_1                   -0.048849      1.000000      0.138170   
bureau_score                    0.005555      0.138170      1.000000   
limit                          -0.026100     -0.388891     -0.003286   
income                          0.015581     -0.156433     -0.002919   
old_open_trd_ms                -0.009316     -0.114167     -0.166021   
no_open_trd                     0.005744      0.032049     -0.292886   
no_trd                         -0.005715     -0.031194     -0.247178   
debt_cap                        0.003299     -0.003115     -0.000061   
basic_max_trd_tnr              -0.024893     -0.127508     -0.157883   
basic_no_trd                   -0.016602     -0.013215     -0.256199   
risk_score_2                   -0.054128      0.445600      0.323836   
risk_score_3                   -0.012163      0.019248      0.00

In [ ]:
corr_matrix = data.corr()
print(corr_matrix)


                       unique_identifier  risk_score_1  bureau_score  \
unique_identifier               1.000000     -0.048849      0.005555   
risk_score_1                   -0.048849      1.000000      0.138170   
bureau_score                    0.005555      0.138170      1.000000   
limit                          -0.026100     -0.388891     -0.003286   
income                          0.015581     -0.156433     -0.002919   
old_open_trd_ms                -0.009316     -0.114167     -0.166021   
no_open_trd                     0.005744      0.032049     -0.292886   
no_trd                         -0.005715     -0.031194     -0.247178   
debt_cap                        0.003299     -0.003115     -0.000061   
basic_max_trd_tnr              -0.024893     -0.127508     -0.157883   
basic_no_trd                   -0.016602     -0.013215     -0.256199   
risk_score_2                   -0.054128      0.445600      0.323836   
risk_score_3                   -0.012163      0.019248      0.00

In [ ]:
plt.figure(figsize=(100, 100))
sns.heatmap(corr_matrix, fmt='10f', cmap='coolwarm', square=True)
plt.title('Correlation Matrix')
plt.show()


In [ ]:
# Count the number of transactions by fraud status
fraud_status_counts = data['default_ind'].value_counts()
print(fraud_status_counts)


default_ind
0    62121
1      363
Name: count, dtype: int64
